In [5]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from PIL import Image

In [6]:
# Images directories
train_dir = '../data/train' 
valid_dir = '../data/valid'
test_dir = '../data/test'

In [7]:
# RESIZE ALL THE IMAGES TO THE MOST COMMUN SIZE

# Resoluciones objetivo para cada clase
target_size_melanoma = (224, 224)
target_size_not_melanoma = (600, 450)

# Ruta base a las carpetas con las imágenes
base_folder = "data"
sets = ['train', 'valid', 'test']
classes = {
    'Melanoma': target_size_melanoma,
    'NotMelanoma': target_size_not_melanoma
}

# Create directories to save resized images
output_base_folder = os.path.join(base_folder, 'resized')
os.makedirs(output_base_folder, exist_ok=True)

for set_name in sets:
    for class_name, target_size in classes.items():
        image_folder = os.path.join(base_folder, set_name, class_name)
        output_folder = os.path.join(output_base_folder, set_name, class_name)
        os.makedirs(output_folder, exist_ok=True)

        for image_file in os.listdir(image_folder):
            img = Image.open(os.path.join(image_folder, image_file))
            img_resized = img.resize(target_size)
            img_resized.save(os.path.join(output_folder, image_file))

print("Todas las imágenes han sido redimensionadas y guardadas en la carpeta 'resized'.")

Todas las imágenes han sido redimensionadas y guardadas en la carpeta 'resized'.


In [15]:
# Generador de datos con normalizacion y aumentación solo para el entrenamiento, esto ayuda a generalizar mejor.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_test_datagen = ImageDataGenerator(rescale=1./255)

# Crear los generadores
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 10682 images belonging to 2 classes.
Found 3562 images belonging to 2 classes.
Found 3561 images belonging to 2 classes.


In [23]:
# Cargar el modelo base preentrenado ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congelar las capas del modelo base
#base_model.trainable = False

# Descongelar más capas para un ajuste fino
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Ajusta este número según la cantidad de capas que quieras descongelar
    layer.trainable = False

# Añadir capas superiores personalizadas
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(2, activation='softmax')(x)  # Suponiendo 2 clases: Melanoma y NotMelanoma

# Crear el modelo completo
model = models.Model(inputs=base_model.input, outputs=outputs)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Mostrar la arquitectura del modelo
model.summary()

# Guardar el mejor modelo
checkpoint = ModelCheckpoint('models/best_melanomaornot_model.keras',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

# Parar temprano si no hay mejora
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)


# Reducir aún más la tasa de aprendizaje
model.compile(optimizer=Adam(learning_rate=1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Reducir la tasa de aprendizaje si no hay mejora
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-7)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_4[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,637,826 (93.99 MB)

 Trainable params: 9,981,442 (38.08 MB)

 Non-trainable params: 14,656,384 (55.91 MB)

In [24]:
# Continuar entrenando el modelo
history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

Epoch 1/10
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5709 - loss: 1.5102
Epoch 1: val_loss improved from inf to 1.50514, saving model to models/best_melanomaornot_model.keras
333/333 ━━━━━━━━━━━━━━━━━━━━ 1242s 4s/step - accuracy: 0.5712 - loss: 1.5100 - val_accuracy: 0.4927 - val_loss: 1.5051 - learning_rate: 1.0000e-06
Epoch 2/10
  1/333 ━━━━━━━━━━━━━━━━━━━━ 15:45 3s/step - accuracy: 0.6875 - loss: 1.3506

2024-08-16 12:20:40.835686: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-08-16 12:20:41.622269: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]



Epoch 2: val_loss did not improve from 1.50514
333/333 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6875 - loss: 1.3506 - val_accuracy: 0.4000 - val_loss: 1.5071 - learning_rate: 1.0000e-06
Epoch 3/10
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7540 - loss: 1.3248
Epoch 3: val_loss improved from 1.50514 to 1.23843, saving model to models/best_melanomaornot_model.keras
333/333 ━━━━━━━━━━━━━━━━━━━━ 1576s 5s/step - accuracy: 0.7540 - loss: 1.3247 - val_accuracy: 0.7998 - val_loss: 1.2384 - learning_rate: 1.0000e-06
Epoch 4/10
  1/333 ━━━━━━━━━━━━━━━━━━━━ 31:51 6s/step - accuracy: 0.8438 - loss: 1.2371

2024-08-16 12:47:03.398872: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-08-16 12:47:05.282274: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]



Epoch 4: val_loss did not improve from 1.23843
333/333 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8438 - loss: 1.2371 - val_accuracy: 0.6000 - val_loss: 1.3011 - learning_rate: 1.0000e-06
Epoch 5/10


...

2024-08-16 13:40:33.421291: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


333/333 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.8750 - loss: 1.1527 - val_accuracy: 0.9000 - val_loss: 1.1097 - learning_rate: 1.0000e-06
Epoch 9/10
333/333 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8159 - loss: 1.2069
Epoch 9: val_loss did not improve from 1.10975
333/333 ━━━━━━━━━━━━━━━━━━━━ 1318s 4s/step - accuracy: 0.8159 - loss: 1.2068 - val_accuracy: 0.8395 - val_loss: 1.1477 - learning_rate: 1.0000e-06
Epoch 10/10
  1/333 ━━━━━━━━━━━━━━━━━━━━ 17:06 3s/step - accuracy: 0.8438 - loss: 1.2116

2024-08-16 14:02:37.778047: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]



Epoch 10: val_loss did not improve from 1.10975
333/333 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8438 - loss: 1.2116 - val_accuracy: 0.7000 - val_loss: 1.2511 - learning_rate: 1.0000e-06
Restoring model weights from the end of the best epoch: 8.


2024-08-16 14:02:38.548148: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [31]:
# EVALUACION

# Cargar el modelo guardado
model = models.load_model('models/best_melanomaornot_model.keras')

# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Recalcular predicciones con ajuste de steps
steps = np.ceil(test_generator.samples / test_generator.batch_size).astype(int)
predictions = model.predict(test_generator, steps=steps)
predicted_classes = np.argmax(predictions, axis=1)

# Verificar longitudes nuevamente
true_classes = test_generator.classes
print(f'Longitud de true_classes: {len(true_classes)}')
print(f'Longitud de predicted_classes: {len(predicted_classes)}')

111/111 ━━━━━━━━━━━━━━━━━━━━ 231s 2s/step - accuracy: 0.9060 - loss: 1.0507
Test Loss: 1.1932311058044434
Test Accuracy: 0.8178490996360779
112/112 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step
Longitud de true_classes: 3561
Longitud de predicted_classes: 3561


In [32]:
# Generar reporte de clasificación
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix")
print(conf_matrix)

              precision    recall  f1-score   support

    Melanoma       0.77      0.91      0.83      1781
 NotMelanoma       0.89      0.72      0.80      1780

    accuracy                           0.82      3561
   macro avg       0.83      0.82      0.82      3561
weighted avg       0.83      0.82      0.82      3561

Confusion Matrix
[[1629  152]
 [ 499 1281]]


In [ ]:
# Cargar el modelo previamente entrenado
model = models.load_model('models/best_melanomaornot_model.keras')

# Descongelar menos capas para un fine tuning más manejable
base_model = model.layers[0]  # Asumiendo que el modelo base es el primer layer
if hasattr(base_model, 'layers'):
    base_model.trainable = True
    for layer in base_model.layers[:-5]:  # Descongelar solo las últimas 5 capas
        layer.trainable = False

# Recompilar el modelo con una tasa de aprendizaje más baja para ajuste fino
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Guardar el mejor modelo durante el ajuste fino
checkpoint = ModelCheckpoint('models/best_melanomaornot_fine_tuned_model.keras',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

# Parar temprano si no hay mejora
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Reducir la tasa de aprendizaje si no hay mejora
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-7)

# Ajustar `steps_per_epoch` para que sea más manejable
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // (2 * train_generator.batch_size),  # Reducir steps_per_epoch
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // (2 * validation_generator.batch_size),  # Reducir validation_steps
    epochs=5,  # Reducir el número de épocas
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

# Evaluar el modelo en el conjunto de pruebas
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

In [41]:
# MODELO CON UN 84% DE ACCURACY

# Calcular el número de steps exactos para cubrir todas las muestras
steps = int(np.ceil(test_generator.samples / test_generator.batch_size))

# Recalcular predicciones para el conjunto de prueba
predictions = model.predict(test_generator, steps=steps)
predicted_classes = np.argmax(predictions, axis=1)

# Validar la longitud de predicciones y etiquetas verdaderas
true_classes = test_generator.classes
print(f'Longitud de true_classes: {len(true_classes)}')
print(f'Longitud de predicted_classes: {len(predicted_classes)}')

# Generar reporte de clasificación si las longitudes coinciden
if len(true_classes) == len(predicted_classes):
    class_labels = list(test_generator.class_indices.keys())
    report = classification_report(true_classes, predicted_classes, target_names=class_labels)
    print(report)

    # Mostrar la matriz de confusión
    conf_matrix = confusion_matrix(true_classes, predicted_classes)
    print("Confusion Matrix")
    print(conf_matrix)
else:
    print("Las longitudes de true_classes y predicted_classes no coinciden.")

112/112 ━━━━━━━━━━━━━━━━━━━━ 248s 2s/step
Longitud de true_classes: 3561
Longitud de predicted_classes: 3561
              precision    recall  f1-score   support

    Melanoma       0.91      0.75      0.83      1781
 NotMelanoma       0.79      0.93      0.85      1780

    accuracy                           0.84      3561
   macro avg       0.85      0.84      0.84      3561
weighted avg       0.85      0.84      0.84      3561

Confusion Matrix
[[1341  440]
 [ 128 1652]]


In [43]:
# Resumen del modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

# Ejemplo de un modelo simple
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Imprimir resumen del modelo
model.summary()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 123008)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │    15,745,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,747,338 (60.07 MB)

 Trainable params: 15,747,338 (60.07 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Interpretación de los resultados:

Precision y Recall:

Melanoma:
Precision: 0.91 - Esto significa que el 91% de las predicciones etiquetadas como "Melanoma" fueron correctas.   

Recall: 0.75 - Esto indica que el modelo identificó correctamente el 75% de todos los casos reales de "Melanoma".   

NotMelanoma:   
Precision: 0.79 - El 79% de las predicciones etiquetadas como "NotMelanoma" fueron correctas.
Recall: 0.93 - El modelo identificó correctamente el 93% de todos los casos reales de "NotMelanoma".   

F1-score:   

Melanoma: 0.83   
NotMelanoma: 0.85   
El F1-score es una medida que equilibra precision y recall, por lo que puntajes cercanos a 1 son buenos.

Accuracy:   

La precisión general del modelo es 0.84, lo que significa que el 84% de las predicciones totales fueron correctas.   

Matriz de Confusión:   

Melanoma:   
1341 casos fueron correctamente clasificados como "Melanoma".
440 casos fueron incorrectamente clasificados como "NotMelanoma".
NotMelanoma:
1652 casos fueron correctamente clasificados como "NotMelanoma".
128 casos fueron incorrectamente clasificados como "Melanoma".
Próximos pasos:
Análisis de errores: Observa los falsos positivos y falsos negativos (las entradas incorrectamente clasificadas) para identificar patrones y entender si hay alguna característica particular que el modelo está pasando por alto.

Más ajuste fino: Puedes continuar ajustando el modelo, probando con diferentes tasas de aprendizaje o descongelando más capas.

Aumento de datos: Considera si podrías mejorar el rendimiento del modelo incrementando la cantidad de datos de entrenamiento mediante técnicas de aumento de datos (data augmentation) o recolectando más datos si es posible.

Experimentación con otros modelos: Aunque ResNet es potente, podrías experimentar con otros modelos de última generación como DenseNet, EfficientNet, o probar variantes de ResNet para ver si ofrecen mejoras adicionales.

Estos resultados son prometedores y sugieren que el modelo está aprendiendo bien, pero aún hay espacio para mejorar, especialmente en la reducción de los falsos positivos y negativos.